In [1]:
import os
import time
import torch

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

from FLTrack.models import ShallowNN
from FLTrack.evals import evaluate_mae_with_confidence
from FLTrack.evals import influence
from FLTrack.evals import evaluate
from FLTrack.evals import euclidean_distance, accumulated_proximity, hessian_eccentricity

features = 169
batch_size = 64
loss_fn = torch.nn.L1Loss()

In [2]:
client_ids = [f"c{i}" for i in range(1, 25)]

In [3]:
def eval(model, loss_fn, dataloader) -> float:
    """
    Evaluate the model with validation dataset.

    Parameters:
    ------------
    model: torch.nn.Module object; model to be evaluated
    loss_fn: torch.nn.Module object; loss function

    Returns:
    ------------
    loss_avg: float; average loss
    """
    batch_loss = []
    for _, (x, y) in enumerate(dataloader):
        outputs = model(x)
        y = y.view(-1, 1)
        loss = loss_fn(outputs, y)
        batch_loss.append(loss.item())
    loss_avg = sum(batch_loss) / len(batch_loss)

    return loss_avg

## Isolated Model Performance

In [103]:
for client in client_ids:
    val_data_path =  "testpt/"+str(client)+".pt"
    val_set = torch.load(val_data_path)
    val_loader = DataLoader(val_set, batch_size, shuffle = True)
    isolated_model_path = "FLTrack/checkpt/isolated/epoch_250/batch256_client_"+str(client)+".pth"
    isolated_model =  ShallowNN(169)
    isolated_model.load_state_dict(torch.load(isolated_model_path))
    print(str(client), eval(isolated_model, loss_fn, val_loader))

c1 1.2270801067352295
c2 1.5948263558935611
c3 1.9564043635671788
c4 1.2360392542261827
c5 1.2223030749489279
c6 1.304645813173718
c7 1.6455158650875092
c8 1.7426149504525321
c9 1.584003562512605
c10 1.264386817165043
c11 1.1598445934218329
c12 1.6015692738925709
c13 1.9488079696893692
c14 1.091790728625797
c15 2.026768694983588
c16 1.697183950079812
c17 1.5474528356602317
c18 2.4321875788948755
c19 1.1305986809176067
c20 1.3184901245178715
c21 1.47362024208595
c22 1.7676244378089905
c23 1.1200449674025825
c24 1.9297072578359533


## Federated Model Performance

In [100]:
global_model = ShallowNN(features)
global_model.load_state_dict(torch.load('FLTrack/checkpt/fedl/epoch_250/25_rounds_10_epochs_per_round/global_model.pth'))

<All keys matched successfully>

In [102]:
for client in client_ids:
    val_data_path =  "testpt/"+str(client)+".pt"
    val_set = torch.load(val_data_path)
    val_loader = DataLoader(val_set, batch_size, shuffle = True)
    print(str(client), eval(global_model, loss_fn, val_loader))

c1 1.4377839334549443
c2 1.5201152284094628
c3 1.534032772887837
c4 1.894552817470149
c5 1.4621374151285957
c6 1.4649371008078258
c7 1.569283813238144
c8 1.6277056523731777
c9 2.0137493506721826
c10 2.017547747363215
c11 1.8707252805297439
c12 1.6075116606319653
c13 2.4495287388563156
c14 1.612745804446084
c15 1.6191439893510606
c16 1.4485814571380615
c17 1.6247947717967786
c18 2.2577746998180044
c19 1.7602314034173654
c20 1.497902689441558
c21 1.4542092660377766
c22 2.5085608618600026
c23 1.9216591612152432
c24 1.6408439521436338


In [104]:
pd.read_csv("FLTrack/training_stats/fedl/fl_stats_epoch1_5/client_c1.csv")

,training_loss,validation_loss
0,54.671667,54.51113


## Error Bars for Federated Learning vs Isolated Training

In [ ]:
eval_list = []
for client in client_ids:
    val_data_path =  "testpt/"+str(client)+".pt"
    val_set = torch.load(val_data_path)
    val_loader = DataLoader(val_set, batch_size, shuffle = True)
    
    isolated_model_path = "checkpt/epoch_500/isolated/batch64_client_"+str(client)+".pth"
    isolated_model =  ShallowNN(features)
    isolated_model.load_state_dict(torch.load(isolated_model_path))
    
    isolated_mae,(iso_lower_band, iso_upper_band), _  = evaluate_mae_with_confidence(isolated_model, val_loader)
    federated_mae,(fed_lower_band, fed_upper_band), _ = evaluate_mae_with_confidence(global_model,val_loader)
    
    eval_dict = {"client_id":client, "Isolated Average MAE": round(isolated_mae, 4),
                 "Isolated MAE lower band":round(iso_lower_band,4),
                 "Isolated MAE upper band":round(iso_upper_band,4),
                 "Federated Average MAE" :round(federated_mae, 4),
                "Federated MAE lower band": round(fed_lower_band, 4),
                "Federated MAE upper band":round(fed_upper_band,4)}
    eval_list.append(eval_dict)
    
eval_df = pd.DataFrame.from_dict(eval_list)
eval_df["clients"] = [i for i in range(1,25)] 

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
bar_width = 0.3
index = eval_df.index

bar1 = ax.bar(index - bar_width / 2, eval_df['Isolated Average MAE'], bar_width, yerr=[
    (eval_df["Isolated Average MAE"] - eval_df["Isolated MAE lower band"]),
    (eval_df['Isolated MAE upper band'] - eval_df["Isolated Average MAE"])
], capsize=5, label='Isolated Model MAE')

bar2 = ax.bar(index + bar_width / 2, eval_df['Federated Average MAE'], bar_width, yerr=[
    (eval_df["Federated Average MAE"] - eval_df["Federated MAE lower band"]),
    (eval_df['Federated MAE upper band'] - eval_df["Federated Average MAE"])
], capsize=5, label='Federated Model MAE')

ax.set_xlabel('Client IDs', fontdict={'fontsize': 13})
ax.set_ylabel("Mean Absolute Error for Validation", fontdict={'fontsize': 13})
ax.set_xticks(index)
ax.set_xticklabels(eval_df['clients'])
ax.legend(fontsize=15, loc="upper right")

# Adjust the xlim to decrease space at the left and right edges
ax.set_xlim(index[0] - 0.7, index[-1] + 0.7)

plt.show()

## Influence with prediction difference at 500 global rounds

In [ ]:
inf_val = []
for client in client_ids:
    model = ShallowNN(features)
    model.load_state_dict(torch.load('checkpt/epoch_500/influence/' + str(client)+ '_fedl_global_500.pth'))
    val_data_path =  "testpt/"+str(client)+".pt"
    val_set = torch.load(val_data_path)
    inf = influence(global_model,model,val_set)
    inf_val.append(round(inf.item(),4))
   
data = {"client id": client_ids, "inf_val": inf_val}
data = pd.DataFrame(data)
data.to_csv("insights/influence_with_pred_diff_ex1.csv" , index=False)

## Influence with prediction difference at 1 global round and 25 local rounds

In [ ]:
global_model = ShallowNN(features)
global_model.load_state_dict(torch.load('checkpt/saving/epoch_500/global_1/global_model.pth'))

inf_val_test = []
for client in client_ids:
    model = ShallowNN(features)
    model.load_state_dict(torch.load('checkpt/epoch_25/influence/1_rounds_25_epochs_per_round/' + str(client)+ '_fedl_global_1_25.pth'))
    val_data_path =  "testpt/"+str(client)+".pt"
    val_set = torch.load(val_data_path)
    inf = influence(global_model,model,val_set)
    inf_val_test.append(round(inf.item(),4))
    
data_test = {"client id": client_ids, "inf_val": inf_val_test}
data_test = pd.DataFrame(data_test)
data_test.to_csv("insights/influence_with_pred_diff_ex2.csv" , index=False)

## Influence with mae at 1 global round and 25 local rounds

In [ ]:
performance_inf = []
for client in client_ids:
    isolated_model = ShallowNN(features)
    isolated_model.load_state_dict(torch.load('checkpt/epoch_500/influence/' + str(client)+ '_fedl_global_500.pth'))
    val_data_path =  "testpt/"+str(client)+".pt"
    val_set = torch.load(val_data_path)
    val_loader = DataLoader(val_set, batch_size, shuffle = True)
    
    _, _ , isolated_mae = evaluate(isolated_model, val_loader, loss_fn)
    _, _ , global_mae = evaluate(global_model,val_loader,loss_fn)
    
    inf = global_mae - isolated_mae
    
    performance_inf.append(round(inf.item(),4))

data = {"client id": client_ids, "inf_val": performance_inf}
data = pd.DataFrame(data)
data.to_csv("insights/influence_with_mae_ex2.csv" , index=False)

## Eccentricity  $\xi^L$ 

$
\begin{equation}\label{eq:Ecc1}
\xi_i = \frac{2\sum_{j=1}^{k}d(\mathcal{H}_{\mu_i}, \mathcal{H}_{\mu_j})}{\sum_{l=1}^{k}\sum _{j=1}^{k} d(\mathcal{H}_{\mu_l}, \mathcal{H}_{\mu_j})},
\end{equation}
$

Where  $\mathcal{H}_{\mu_i}$ is the Hessian matrix of client $i$, and $d(.,.)$ is the Euclidean distance between the Hessian matrices of two clients. 

In [ ]:
local_matrix_dict = {
        key: torch.load("hessians/epoch_500/iso/" + str(key) + ".pth")
        for key in client_ids
    }

In [ ]:
ecce_dict = hessian_eccentricity(local_matrix_dict, euclidean_distance)
ecc_hessian = {"client id": client_ids, "hess_ecc": list(ecce_dict.values())}
ecc_hessian = pd.DataFrame(ecc_hessian)
ecc_hessian.to_csv("insights/eccentricity_with_hessian_euclidean_with_local_model.csv" , index=False)

## Eccentricity $\xi^G$ 

$
\begin{equation}\label{eq:Ecc1}
\xi_i = \frac{2\sum_{j=1}^{k}d(\mathcal{H}^i_{\mathcal M}, \mathcal{H}^j_{\mathcal M})}{\sum_{l=1}^{k}\sum _{j=1}^{k} d(\mathcal{H}^l_{\mathcal M}, \mathcal{H}^j_{\mathcal M})},
\end{equation}
$

where $\mathcal{H}^i_{\mathcal M}$ is the Hessian matrix of client $i$, and $d(.,.)$ is the Euclidean distance between the Hessian matrices global with respect to the validation dataset of clients $i$. 

In [ ]:
global_matrix_dict = {
        key: torch.load("hessians/epoch_500/fed/" + str(key) + ".pth")
        for key in client_ids
    }

In [ ]:
ecce_dict_g = hessian_eccentricity(global_matrix_dict, euclidean_distance)
ecc_hessian_g = {"client id": client_ids, "hess_ecc": list(ecce_dict_g.values())}
ecc_hessian_g = pd.DataFrame(ecc_hessian)
ecc_hessian_g.to_csv("insights/eccentricity_with_hessian_euclidean_with_globall_model.csv" , index=False)

In [ ]:
[0.022619629561823677, 1.62271054254116, -0.5735217639598182, 0, -0.7876090943058588, 0, 0.43482986006497976, 1.6028013361292264, -0.10931628063829524, -0.6966764002778227, -0.17369175360870828, 0.16841961315934875, -0.015073389215806005, -0.4861588087040261, -0.06287855664828562, -0.23575240856089766, 0.07736005847409497, 0.7842280658106887, 0.12698984913067995, -0.08667967230145918, 0.15672320636514706, 0.13853950906135654, 0.7912020259738062, 0.6457963290841103]